In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
from urllib import request
from PIL import Image
import matplotlib.pyplot as plt
from io import BytesIO

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
url = "https://www.jumia.co.ke/all-products/"
requests = requests.get(url)
## Parsing the html soup
soup = BeautifulSoup(requests.text,"html.parser")
items = soup.find_all("article", attrs = {"class":"prd _fb col c-prd"})

In [3]:
print(len(items))

40


In [4]:
print(items[0])

<article class="prd _fb col c-prd"><a class="btn _i _rnd -mas -fsh0 -me-start _wslt _sec" data-pop-open="addToWishlist" data-pop-trig="atw" data-sku="NI534ST019V71NAFAMZ" href="/customer/account/login/?tkWl=NI534ST019V71NAFAMZ-120099954&amp;return=%2Fall-products%2F" rel="nofollow"><svg aria-label="Add to wishlist" class="ic -f-or5" height="16" viewbox="0 0 24 24" width="16"><use xlink:href="https://www.jumia.co.ke/assets_he/images/i-icons.a66628fd.svg#saved-items"></use></svg></a><a class="core" data-gtm-brand="NIVEA" data-gtm-category="Health &amp; Beauty/Beauty &amp; Personal Care/Personal Care/Skin Care/Body/Moisturizers/Lotions" data-gtm-dimension23="" data-gtm-dimension26="630" data-gtm-dimension27="4.7" data-gtm-dimension28="1" data-gtm-dimension37="0" data-gtm-dimension43="BDEAL|BLF|BLF_02|Boost|Bundle|COL_224|COL_260|COL_295|COL_297|COL_302|CP_24|CP_26|CP_MT117|CP_MT123|CP_MT134|CP_UN10|CP_UN105|CP_UN138|CP_UN237|CP_UN247|CP_UN250|CP_UN255|CP_UN34|CP_UN9|DOW_10|DOW_100|INV_CLE

In [5]:
first_items = items[0]
first_items_name =first_items.find("h3",attrs = {"class":"name"}).text
print(first_items_name)

NIVEA Radiant & Beauty Advanced Care Lotion For Women - 400ml (Pack Of 2)


In [6]:
first_items_price = first_items.find("div", attrs = {"class":"prc"}).text
first_items_price

'KSh 854'

In [7]:
first_items = items[0]
first_items_link = first_items.find("a", attrs = {"class":"core"}).get("href")
print(first_items_link)

/nivea-radiant-beauty-advanced-care-lotion-for-women-400ml-pack-of-2-94993120.html


first_items = items[0]
first_items_image = first_items.find("img", attrs = {"class":"img"}).get("image_url")
print(first_items_image)

image = Image.open(first_items_image)
response = requests.get(first_items_image)

image = Image.open(BytesIO(response.content))

plt.imshow(image)
plt.axis('off')
plt.show()

In [8]:
##discount
first_items = items[0]
first_items_discount = first_items.find("div", attrs = {"class":"bdg _dsct _sm"}).text
print(first_items_discount)

42%


In [9]:
records= []
for item in items:
    try:
        item_name =item.find("h3",attrs = {"class":"name"}).text
        current_price =item.find("div", attrs = {"class":"prc"}).text
        old_price_element = item.find("div", attrs = {"class":"old"})
        item_discount_element = item.find("div", attrs = {"class":"bdg _dsct _sm"})
        item_link = item.find("a", attrs = {"class":"core"}).get("href")

        if old_price_element: 
            old_price = old_price_element.text
        else:
            old_price = None
        if item_discount_element:
            item_discount = item_discount_element.text
        else:
            item_dicount = None
        records.append(( item_name, current_price,old_price, item_discount, item_link))

    except AttributeError:
        records.append(None)
print(len(records))

records

40


[('NIVEA Radiant & Beauty Advanced Care Lotion For Women - 400ml (Pack Of 2)',
  'KSh 854',
  'KSh 1,460',
  '42%',
  '/nivea-radiant-beauty-advanced-care-lotion-for-women-400ml-pack-of-2-94993120.html'),
 ('Vitron HTC4388FS - 43" Smart  Android Frameless TV - Black',
  'KSh 21,506',
  'KSh 28,599',
  '25%',
  '/vitron-htc4388fs-43-smart-android-frameless-tv-black-93348943.html'),
 ("Hikers 43'' Inch Frameless Android Smart FHD LED TV - Black",
  'KSh 21,599',
  'KSh 28,644',
  '25%',
  '/hikers-43-inch-frameless-android-smart-fhd-led-tv-black-130121825.html'),
 ('NIVEA Perfect & Radiant Even Tone Day And Night Cream For Women - 50ml',
  'KSh 899',
  'KSh 1,560',
  '42%',
  '/nivea-perfect-radiant-even-tone-day-and-night-cream-for-women-50ml-68528030.html'),
 ('AILYONS FK-0301 Stainless Steel 1.8L Electric Kettle-Black',
  'KSh 693',
  'KSh 1,399',
  '50%',
  '/ailyons-fk-0301-stainless-steel-1.8l-electric-kettle-black-34080919.html'),
 ('Nunix R5 Hot & Normal Free Standing Water Dispe

In [10]:
import time
import requests
from bs4 import BeautifulSoup

base_url = "https://www.jumia.co.ke/all-products/?page={}#catalog-listing"
records = []

for page_number in range(1, 51):
    url = base_url.format(page_number)
    time.sleep(5)
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        print(f"Page {page_number} scrapped successfully")
        items = soup.find_all("article", attrs={"class": "prd _fb col c-prd"})
        for item in items:
            try:
                item_name = item.find("h3", attrs={"class": "name"}).text
                current_price = item.find("div", attrs={"class": "prc"}).text
                old_price_element = item.find("div", attrs={"class": "old"})
                item_discount_element = item.find("div", attrs={"class": "bdg _dsct _sm"})
                item_link = item.find("a", attrs={"class": "core"}).get("href")
                
                if old_price_element:
                    old_price = old_price_element.text
                else:
                    old_price = None
                if item_discount_element:
                    item_discount = item_discount_element.text
                else:
                    item_discount = None
                records.append((item_name, current_price, old_price, item_discount, item_link))
            except AttributeError:
                records.append(None)
    else:
        print(f"Page {page_number} not scrapped")

Page 1 scrapped successfully
Page 2 scrapped successfully
Page 3 scrapped successfully
Page 4 scrapped successfully
Page 5 scrapped successfully
Page 6 scrapped successfully
Page 7 scrapped successfully
Page 8 scrapped successfully
Page 9 scrapped successfully
Page 10 scrapped successfully
Page 11 scrapped successfully
Page 12 scrapped successfully
Page 13 scrapped successfully
Page 14 scrapped successfully
Page 15 scrapped successfully
Page 16 scrapped successfully
Page 17 scrapped successfully
Page 18 scrapped successfully
Page 19 scrapped successfully
Page 20 scrapped successfully
Page 21 scrapped successfully
Page 22 scrapped successfully
Page 23 scrapped successfully
Page 24 scrapped successfully
Page 25 scrapped successfully
Page 26 scrapped successfully
Page 27 scrapped successfully
Page 28 scrapped successfully
Page 29 scrapped successfully
Page 30 scrapped successfully
Page 31 scrapped successfully
Page 32 scrapped successfully
Page 33 scrapped successfully
Page 34 scrapped su

In [11]:
# Save records to CSV file
import csv
csv_file = "jumia_products.csv"
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Item Name", "Current Price", "Old Price", "Discount", "Item Link"])
    writer.writerows(records)

print("Data saved to", csv_file)

Data saved to jumia_products.csv


In [12]:
df = pd.read_csv("jumia_products.csv")

In [13]:
print("First few rows of the DataFrame:")
df.head(5)

First few rows of the DataFrame:


,Item Name,Current Price,Old Price,Discount,Item Link
0,NIVEA Radiant & Beauty Advanced Care Lotion Fo...,KSh 854,"KSh 1,460",42%,/nivea-radiant-beauty-advanced-care-lotion-for...
1,"Vitron HTC4388FS - 43"" Smart Android Frameles...","KSh 21,506","KSh 28,599",25%,/vitron-htc4388fs-43-smart-android-frameless-t...
2,Hikers 43'' Inch Frameless Android Smart FHD L...,"KSh 21,599","KSh 28,644",25%,/hikers-43-inch-frameless-android-smart-fhd-le...
3,NIVEA Perfect & Radiant Even Tone Day And Nigh...,KSh 899,"KSh 1,560",42%,/nivea-perfect-radiant-even-tone-day-and-night...
4,AILYONS FK-0301 Stainless Steel 1.8L Electric ...,KSh 693,"KSh 1,399",50%,/ailyons-fk-0301-stainless-steel-1.8l-electric...


In [14]:
print("\nSummary statistics:")
df.describe().T


Summary statistics:


,count,unique,top,freq
Item Name,2000,1847,Motorola Moto Edge Plus Leather Phone Case,11
Current Price,2000,1105,KSh 699,34
Old Price,1819,1036,"KSh 1,500",30
Discount,1819,59,50%,390
Item Link,2000,1891,/generic-strap-for-apple-watch-7-45mm-6se54-44...,3


In [15]:
print("\nData types of columns:")
df.dtypes


Data types of columns:


Item Name        object
Current Price    object
Old Price        object
Discount         object
Item Link        object
dtype: object

In [16]:
print("\nMissing values:")
df.isnull().sum()


Missing values:


Item Name          0
Current Price      0
Old Price        181
Discount         181
Item Link          0
dtype: int64